http://localhost:8889/notebooks/git/google-quest-challenge/working/QstBertBase0113_1.ipynb

no freezing of bert model

# start

In [1]:
trntmstmp=1584412344
valtmstmp=1585069785
import datetime
print([datetime.datetime.fromtimestamp(o).strftime('%Y-%m-%d %H:%M:%S') for o in (trntmstmp, valtmstmp)])

grand_total=1.5e8
MIN_TM_TRN=1580947200
MIN_TM_TST=1581552000
print([datetime.datetime.fromtimestamp(o).strftime('%Y-%m-%d %H:%M:%S') for o in (MIN_TM_TRN, MIN_TM_TST)])


CHNKSZ=1e6
POST_RATE_WANTED=0.1

['2020-03-17 02:32:24', '2020-03-24 17:09:45']
['2020-02-06 00:00:00', '2020-02-13 00:00:00']


# setup

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)
import datetime
def dtnow(): return datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

SEED=101
HOME='/data/git/recsys20'
p_in=f'{HOME}/input'

import torch
from transformers import *
import torch
device=torch.device('cpu')
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader


pretrained_weights='bert-base-multilingual-cased'
bertmodel = BertModel.from_pretrained(pretrained_weights, output_hidden_states=True)
tokenizer = BertTokenizer.from_pretrained(pretrained_weights, do_lower_case=False)


In [3]:
cols=[
'toks',
'hshtgs',
'twtid',
'media',
'links',
'domns',
'twttyp',
'lang',
'tm',

'u1id',
'u1_fllwer_cnt',
'u1_fllwng_cnt',
'u1_vrfed',
'u1_create_tm',

'u2id',
'u2_fllwer_cnt',
'u2_fllwng_cnt',
'u2_vrfed',
'u2_create_tm',

'u1_fllw_u2',
'reply_tm',
'retwt_tm',
'retwt_cmmnt_tm',
'like_tm',
]
cols_cat = ['twttyp','lang']
cols_val = cols[:-4]
cols_tgt_tmstmp=[
    'retwt_tm',
    'reply_tm',
    'like_tm',
    'retwt_cmmnt_tm',
]
cols_tgt=[o.split('_tm')[0] for o in cols_tgt_tmstmp]
tgts             = ['Retweet','Reply','Like','RTwCmnt',]
assert cols_tgt == ['retwt',  'reply','like','retwt_cmmnt',]
ntgts=len(tgts)


tgt2col=dict(zip(tgts,cols_tgt))
tgt2col

{'Retweet': 'retwt',
 'Reply': 'reply',
 'Like': 'like',
 'RTwCmnt': 'retwt_cmmnt'}

# prep

In [4]:
%%time
dftrn=pd.read_csv(
    f'{p_in}/trn_{trntmstmp}.tsv',
    sep='\x01', header=None, 
    encoding='utf-8', 
    names=cols, 
    nrows=1e4
)

CPU times: user 100 ms, sys: 39.8 ms, total: 140 ms
Wall time: 142 ms


In [5]:
# tokenizer.pad_token, tokenizer.pad_token_id
# ('[PAD]', 0)

# tokenizer.sep_token, tokenizer.sep_token_id
# ('[SEP]', 102)
maxlen=512
def mkids(x):
    tokids=list(map(int, x.split('\t')))
    l=len(tokids) 
    if l<=maxlen: 
        return tokids + [0]*(maxlen-len(tokids))
    else: 
        return tokids[:maxlen-1]+[102]

In [6]:
def mk_tensors(df, istrn=True):
    tokids=dftrn.toks.apply(lambda x: mkids(x))
    Xarr=np.array(list(tokids))
    X=torch.tensor(Xarr,dtype=torch.long)
    if not istrn: return X
    ys=dftrn[cols_tgt_tmstmp].notna().values
    ys=torch.tensor(ys,dtype=torch.float)
    return X,ys
    

In [7]:
X,ys = mk_tensors(dftrn)

In [8]:
BS=8
ds = TensorDataset(X,ys)
dl = DataLoader(ds, batch_size=BS, shuffle=True)
for step, batch in enumerate(dl):
    X_b,ys_b = (o.to(device) for o in batch)
    print(X_b.shape,ys_b.shape)
    break

torch.Size([8, 512]) torch.Size([8, 4])


# model

In [9]:
EPOCHS=1
WD=0.01
LR=1e-5
SCHDLR_FUNC = get_cosine_schedule_with_warmup 
WARMUP_RATE = 0.05
N_CYCLS = .5
EPS = 1e-6


In [10]:
BS=16
ds = TensorDataset(X,ys)
dl = DataLoader(ds, batch_size=BS, shuffle=True)
for step, batch in enumerate(dl):
    X_b,ys_b = (o.to(device) for o in batch)
    print(X_b.shape,ys_b.shape)
    break

torch.Size([16, 512]) torch.Size([16, 4])


In [11]:
# https://huggingface.co/transformers/model_doc/bert.html#bertmodel
bertmodel=bertmodel.eval()
with torch.no_grad():
    last_hidden_state, pooler_output, hidden_states = bertmodel(X_b,)
    avg_pool = torch.mean(last_hidden_state,1)
    max_pool,_ = torch.max(last_hidden_state,1)
last_hidden_state.shape,pooler_output.shape, len(hidden_states), avg_pool.shape, max_pool.shape

(torch.Size([16, 512, 768]),
 torch.Size([16, 768]),
 13,
 torch.Size([16, 768]),
 torch.Size([16, 768]))

In [12]:
# see RobertaClassificationHead in transformers/modeling_roberta.py
N_HIDDEN = 768 
class TwtModel(nn.Module):
    def __init__(self, bertmodel, num_labels=4):
        super(TwtModel, self).__init__()
        self.bertmodel = bertmodel
#         for param in self.bertmodel.parameters():
#             param.requires_grad=False
        
        dense_size = 64
        self.relu=nn.ReLU(inplace=True)
        nx = N_HIDDEN*2
        self.dense = nn.Linear(nx, nx)
        self.dropout = nn.Dropout(0.1)
        self.out_proj = nn.Linear(nx, num_labels)
        
    def forward(self, x):
        attn_msk = (x!=tokenizer.pad_token_id).float()
        segments = torch.zeros(x.shape, dtype=torch.long)
        last_hidden_state, pooler_output, hidden_states = self.bertmodel(x, 
                                                                         attention_mask=attn_msk,
                                         token_type_ids=segments)
        avg_pool = torch.mean(last_hidden_state,1)
        max_pool,_ = torch.max(last_hidden_state,1)
        x = torch.cat([avg_pool,max_pool],1) 
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x

In [13]:
model=TwtModel(bertmodel)
model = model.to(device)

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': WD},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

optimizer = AdamW(optimizer_grouped_parameters, lr=LR, eps=EPS)
t_total = int(EPOCHS*len(dl))
scheduler = SCHDLR_FUNC(
    optimizer, 
    num_warmup_steps=int(WARMUP_RATE*t_total), 
    num_training_steps=t_total,
    num_cycles=N_CYCLS,
)


In [14]:
for epoch in range(EPOCHS):
    print(dtnow(), f'epoch {epoch} starts')
    y_tr_epoch = []
    y_pred_tr_epoch = []
    for step, batch in enumerate(dl):
        model.train()
        x_b,ys_b = (o.to(device) for o in batch)
        yb_pred = model(x_b)

        loss =  F.binary_cross_entropy_with_logits(yb_pred,ys_b)
        loss.backward()
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        model.zero_grad()
        print(loss.item())


2020-03-26 15:22:08 epoch 0 starts
0.669920802116394
0.7151994705200195
0.7220084071159363
0.6968874931335449
0.6777797937393188
0.6921109557151794
0.6809502840042114
0.6890968084335327
0.6476747393608093
0.6406773328781128
0.646051287651062
0.6112195253372192
0.6161973476409912
0.5813122987747192
0.563540518283844
0.5503049492835999
0.5376545190811157
0.500637948513031
0.485745370388031
0.4646286368370056
0.4568083584308624
0.44872015714645386
0.42671632766723633
0.3812297582626343
0.4689415693283081
0.42789602279663086
0.3591653108596802
0.34643855690956116
0.3629245162010193
0.33644336462020874
0.38637787103652954
0.31037163734436035
0.3333966135978699
0.3493550717830658
0.3682388961315155
0.2514880299568176
0.3200189769268036
0.2738972306251526
0.3205811679363251
0.32253456115722656
0.295433908700943
0.3095294237136841
0.3666035234928131
0.38536328077316284
0.25097107887268066
0.3411667048931122
0.4056805968284607
0.2872089147567749
0.2606630325317383
0.2532269060611725
0.255089104

KeyboardInterrupt: 